<a href="https://colab.research.google.com/github/daicamino/lecture-ai-engineering/blob/master/PJT04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JPT04 講義資料の質疑応答のまとめ機能の作成

神野大輔

##はじめに

　本プログラムは、『講義内容質問フォーム（回答）』の質問を要約したうえで並び替え、講師が類似した質問をまとめて回答することを支援する質問要約ツールである。

　生成モデルは、本講座の第３回演習のプログラムを引用しLlama3を利用している。また、それ以外のコードのドラフトはGeminiにより生成した。

　「質問」または「要約」をもとに類似度によるグループ分けを試みたが、分類の精度が上げられなかった。また、要約をソートすることでもある程度視認性が改善できるため、質問の内容による分類は不採用とした。

　加えて、処理中にも新しい質問が追加される可能性があるため、要約のソートは手動で行うこととする。

　なお、モデルのダウンロードには時間を要するため、本プログラムの利用時には初期設定までは事前に完了していることが望ましい。



##取扱方法

1.   hugging faceのトークンを準備ください。
2.   『講義内容質問フォーム（回答）』のB列に「要約」を追加ください。もしも、プログラム実行時に該当の列がないばあいは、「要約」列が最後列に自動で追加されます。
3.   『講義内容質問フォーム（回答）』を、Google スプレッドシート形式で保存してください。
4.   このノートブックを「講義質問フォーム」と同じフォルダに格納
してください。
5.   処理完了後、『講義内容質問フォーム（回答）』を開き、手動にて「要約」列でソートしてください。

# 初期設定

##環境設定
ご自身の環境に合わせて変更ください。

In [ ]:
# 作業ディレクトリとQAファイル名の指定
work_dir = '/content/drive/MyDrive/Colab Notebooks/lecture-ai-engineering/PJT04/'
QAsheet = '講義内容質問フォーム（回答）'

# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import random
random.seed(0)

## Goodle Driveへの接続と各種認証

In [ ]:
# Goodle Driveへの接続
from google.colab import drive
drive.mount('/content/drive')

# colab Notebookの認証
from google.colab import auth
auth.authenticate_user()

# gspreadの認証
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

## LLMモデルの読み込み

　本講義の第３回演習のプログラムより引用。


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# モデル(Llama3)の読み込み

!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
)

#処理

##GSpreadの読み込み

In [ ]:
import pandas as pd

worksheet = gc.open(QAsheet).sheet1
rows = worksheet.get_all_values()
qa_data = pd.DataFrame(rows[1:], columns=rows[0])

# 「要約」列が存在しない場合は追加
if '要約' not in qa_data.columns:
    qa_data['要約'] = ''
    # スプレッドシートにも列を追加 (ヘッダー行)
    worksheet.insert_cols([['要約']], col=len(rows[0]) + 1)

# prompt: qa_dataの内容を表示
qa_data.head()

##質問の要約

In [ ]:
# prompt: もしも'要約'が空白のとき、各行の質問を要約して、ワークシートの'要約'列に挿入する。

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 各行の「要約」列が空白かどうかを確認し、空白の場合は要約を生成して挿入
for index, row in qa_data.iterrows():
    if row['要約'] == '':
        question = row['質問']
        messages = [
            {"role": "system", "content": "コンテンツの件名を30文字以内で生成する。コンテンツに記載されていないことは表示しない。日本語で回答する。"},
            {"role": "user", "content": f"コンテンツ：{question}"},
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        summary_output = model.generate(
            input_ids,
            max_new_tokens=25, # 要約のトークン数
            eos_token_id=terminators,
            do_sample=False,
        )

        summary_tokens = summary_output[0][input_ids.shape[-1]:]
        summary_str = tokenizer.decode(summary_tokens, skip_special_tokens=True)
        summary_str = summary_str.replace('タイトル：', '').replace('件名：', '').replace('「', '').replace('」', '')

        # DataFrameの「要約」列を更新
        qa_data.at[index, '要約'] = summary_str

        # スプレッドシートの該当セルを更新
        # ヘッダー行があるため、DataFrameのインデックスに2を加える (0始まりのDataFrameインデックス + 1 (ヘッダー) + 1 (スプレッドシートは1始まり))
        # 「要約」列のインデックスを取得
        summary_col_index = qa_data.columns.get_loc('要約') + 1 # gspreadは1始まり

        worksheet.update_cell(index + 2, summary_col_index, summary_str)

print("要約の更新が完了しました。")
print(qa_data[['質問', '要約']].head()) # 更新されたデータの確認